In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from policy import Policy
from langchain_community.embeddings import OllamaEmbeddings

policy = Policy(data_dir='storage/policy', embedding=OllamaEmbeddings(model='llama3'), k=5)
tools = policy.get_tools()
tools

{'lookup_policy_tool': LookupPolicyTool(policy=<policy.Policy object at 0x000002E97AD154C0>)}

In [3]:
print(tools['lookup_policy_tool'].invoke(input={'query': "حداکثر برای چند نفر میشه بلیط تهیه کرد؟"}))

FAQ_SUBJECT: قطار
FAQ_QUESTION:
در یک خرید، برای چند مسافر به‌صورت همزمان می‌توانم خرید کنم؟
FAQ_ANSWER:
<p>حداکثر بلیت قابل ارائه به هر کاربر غیر حضوری در هر خرید برای قطار های اتوبوسی و 4 تخته تعداد 4 بلیت ، قطار های 6 تخته تعداد 6 بلیت و واگن حمل خودرو تعداد 1 بلیت میباشد .</p><p><br>&nbsp;</p>

FAQ_SUBJECT: پرواز خارجی
FAQ_QUESTION:
خرید بلیط به‌صورت  گروهی چه شرایطی دارد؟
FAQ_ANSWER:
<p style="text-align:right;">برای مسافران بیشتر از 20 نفر می‌توانید با مرکز پشتیبانی تماس بگیرید و اطلاعات پرواز و مسافران خود را اعلام کنید تا درخواست خرید گروهی شما بررسی و بلیط‌ها تهیه شود.</p><ul><li style="text-align:justify;"><a href="tel:+982143900000">۰۲۱-۴۳۹۰۰۰۰۰ : شماره تماس</a></li></ul>

FAQ_SUBJECT: هتل داخلی
FAQ_QUESTION:
چه ساعتی می‌توانم اتاق را تحویل بگیرم و چه ساعتی باید اتاق را تحویل بدهم؟
FAQ_ANSWER:
<p style="text-align:right;">معمولا در اکثر هتل ها ساعت ورود 14 و خروج در ساعت 12 انجام میگردد .&nbsp;</p>

FAQ_SUBJECT: هتل خارجی
FAQ_QUESTION:
در رزرو هتل خارجی ،کودکان تا چه سنی رای